In [ ]:
from ipypublish.scripts import nb_setup

In [ ]:
# define the subject to report: subject selection - a list with reportable files is generated
%pylab inline

import os, os.path
import json
from os.path import join as opj

import warnings
warnings.filterwarnings("ignore")

from IPython.display import SVG, Markdown, display, HTML, Image, clear_output
START = '\033[4m' # start underline
END = '\033[0m' # stop underline

import matplotlib.image as mpimg

from datetime import datetime

from pdf2image import convert_from_path

import shutil

import csv

clear_output()

import imageio
from PIL import Image

import IPython

In [ ]:
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

In [ ]:
# Define some parameters
wDir = '/data/Patients'
oDir = '/data/output'
iDir = '/data/Patients/SubjectInfoFiles'
patdcmroot = '/data/Patients/tmp_dicom'
bids_source = '/data/BIDS_source'
# folders and files in the subject directory
#folders = ['anat', 'func', 'dwi', 'results', 'fmap', 'not_used']
# dictionary linking SeriesDescription to folders
#f_dict = {'T1_mprage':'anat', 'WGEN':'func', 'READ':'func', 'ENCO_OLD':'func', 'ENCO_NEW':'func', 'DTI':'dti'} 
# dictionary defining the number of imags in a functional run, if lower, the run is not complete, double values for SMS (PRISMA)
#fimg_dict = {'WGEN': [120, 240], 'READ':[120, 240], 'ENCO_OLD':[96, 192], 'ENCO_NEW':[360,720]}

drempels_dict = {'wgen': 'T>5', 'read': 'T>3.5', 'enco_old': 'T>3.5', 'enco_new': 'T>2.5', 'handen': 'T>7', 'voeten': 'T>7', 'mond': 'T>7', 'visus': 'T>10' }

In [ ]:
# Subject selection:
def subjects4report():
    """
    
    Screens all entrys with sub-<#> in wDir for completion of datatreatment
    
    returns: patient_list = [subdir, [patientinfo = dcm tags], {figures info per task}]
        
    """
        
    os.chdir(wDir)
    
    # Here we define the list of subjects in wDir: sub_list
    sub_list = []
    print('Checking %s' % wDir) 
    for item in os.listdir(wDir):
        if item.startswith('sub-'):
            sub_list.append(item)
        
    sub_list = sorted(sub_list)
    
    # Create a list 'patient_list' with for every patient the following elements:
    # [[ sub-<accession#>, [dcmtags list], {1stlevel task figures dict}], [idem], ....]
    
    patient_list = []
    
    for i, subdir in enumerate(sub_list):
        # subdir is the first element of every entry in patient_list
        sub_list = [] # temp list to append to patient_list 
        sub_list.append(subdir)
        firstlevel_list = [] # read it from the json if any
        patientinfo =[] # this is the second element of every entry in patient_list 
        
        jsonfile = opj(wDir, 'SubjectInfoFiles', subdir + '_log.json')
        
        try:
            with open(jsonfile, "r") as j_file:
                jdata = json.load(j_file)
            
            firstlevel_list = sorted(list(set(jdata["Statistics"])))
            try:
                patientinfo = jdata["Subject DCM tags"]
            except:
                patientinfo = ['No DCM tags!']
        except:
            patientinfo = ['No json file!']
            
        sub_list.append(patientinfo)
        
        firstlevelfig_dict = {} 
        # this will become a dictionary with elements { task: [number of figures list] }
        # task will be appended with '_N' if the figures are from normalized data
        # the list with numbers [a, b, c, d, e] where
        # a = the number of realignmentplot images for the task
        # b = the number of DesignMatrix images for the task
        # c = the number of GlassBrain functional images for the task
        # d = the number of detrended signal curve images for the task
        # e = the number of anatomy or EPI functional overlay images for the task
        
        
        # define the figures dirs, normalized and not
        if len(firstlevel_list) != 0:
            fig_dir = opj(oDir, 'results_' + subdir, 'figures')
            norm_fig_dir = opj(oDir, 'results_' + subdir + '-N', 'figures')
                    
            # check the figures for each task in firstlevel_list, normalised
            if os.path.isdir(norm_fig_dir):
                for task in firstlevel_list:
                    
                    DesignMatrix_figs = 0
                    GlassBrain_figs = 0
                    detrended_figs = 0
                    overlay_figs = 0
                    realignmentplot_figs = 0
                    
                    for ffile in os.listdir(norm_fig_dir): # count the figures
                        if 'DesignMatrix' in ffile and task in ffile:
                            DesignMatrix_figs =+ 1
                        if 'GlassBrain' in ffile and task in ffile:
                            GlassBrain_figs =+ 1
                        if 'detrended' in ffile and task in ffile:
                            detrended_figs =+ 1
                        if 'overlay' in ffile and task.upper() in ffile:
                            overlay_figs =+ 1
                        if 'realignmentplot' in ffile and task in ffile:
                            realignmentplot_figs =+ 1
                
                    firstlevelfig_dict[task + '-N'] = [realignmentplot_figs, DesignMatrix_figs, GlassBrain_figs, detrended_figs, overlay_figs]
                
            # check the figures for each task in firstlevel_list, not normalised
            if os.path.isdir(fig_dir):
                for task in firstlevel_list:

                    DesignMatrix_figs = 0
                    GlassBrain_figs = 0
                    detrended_figs = 0
                    overlay_figs = 0
                    realignmentplot_figs = 0

                    for ffile in os.listdir(fig_dir): # count the figures
                        if 'DesignMatrix' in ffile and task in ffile:
                            DesignMatrix_figs =+ 1
                        if 'GlassBrain' in ffile and task in ffile:
                            GlassBrain_figs =+ 1
                        if 'detrended' in ffile and task in ffile:
                            detrended_figs =+ 1
                        if 'overlay' in ffile and task.upper() in ffile:
                            overlay_figs =+ 1
                        if 'realignmentplot' in ffile and task in ffile:
                            realignmentplot_figs =+ 1

                    firstlevelfig_dict[task] = [realignmentplot_figs, DesignMatrix_figs, GlassBrain_figs, detrended_figs, overlay_figs]
                    
                         
            sub_list.append(firstlevelfig_dict)
        
        # append for each subject as below
        patient_list.append(sub_list)
        
        #print('There are % subjects in %s.' % (str(len(patient_list))), wDir)
    
    clear_output()
        
    return patient_list

In [ ]:
def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))

In [ ]:
def select_patient4report(patient_list):
    """
    From the 'patient-list' defined with subject4report(), select one for reporting
    
    The list to select from is constrained with certain criteria that can be selected
    
    """
    
    print('Which subject should I list to select from?')
    print('A: All subjects in %s.' % wDir)
    print('B: Only subjects with one or more 1st-level processed tasks?')
    print('C: Only those of B with normalised processing?')
    print('D: Subjects without tasks if any.')
    print('')
    
    answer = input('Make a selection (default = A): ')
    
    # Generate a list according to the selection for presentation above
    newpat_list = []

    if answer == '': answer = 'A'
    
    if answer == 'A':
        newpat_list = patient_list
    
    elif answer == 'B':
        for i, subject in enumerate(patient_list):
            if subject[2] != {}:
                newpat_list.append(subject)
                
    elif answer == 'C':
        for i, subject in enumerate(patient_list):
            taskkeylist = list(subject[2].keys())
            if any('-N' in s for s in taskkeylist):
                newpat_list.append(subject)
                
    elif answer == 'D':
        for i, subject in enumerate(patient_list):
            if subject[2] == {}:
                newpat_list.append(subject)  
    
    # print all the lines in newpat_list according to selection
    for i, subject in enumerate(newpat_list):

        allfigtasks = []
        fig_dict = subject[2]
        for item in fig_dict.keys():
            psum = sum(fig_dict[item])
            if psum >= 5:
                allfigtasks.append(item)


        if len(allfigtasks) == 0:
            allfigtasks = ['No Tasks for reporting']
            
        if type(subject[1]) == list:

            if hasNumbers(subject[1][0]):
                sub_name = subject[1][3]
                sub_adrema = subject[1][0]
                sub_date = subject[1][2]    
            else:
                sub_name = subject[1][0]
                sub_adrema = subject[1][1]
                sub_date = subject[1][4]
                
        if type(subject[1]) == dict:
            sub_name = subject[1]['Patient Name']
            sub_adrema = subject[1]['Patient ID']
            sub_date = subject[1]['Study Date']
            

        print("%i. %s (adrema %s) - Exam date: %s - Tasks with all figures: %s" %(i+1, sub_name, sub_adrema, sub_date, allfigtasks))

    # Make a selection for structured report except when option 'D' was selected
    if answer in ['A', 'B', 'C']:
        numpat = int(input('Indicate the number of the patient to report: '))

        patient2report = newpat_list[numpat-1]

        taskList = list(patient2report[2].keys())

        if type(newpat_list[numpat-1][1]) == list:
            for item in newpat_list[numpat-1][1]:
                if hasNumbers(newpat_list[numpat-1][1][0]):
                    sub_name = newpat_list[numpat-1][1][3]
                    sub_adrema = newpat_list[numpat-1][1][0]
                    sub_date = newpat_list[numpat-1][1][2]    
                else:
                    sub_name = newpat_list[numpat-1][1][0]
                    sub_adrema = newpat_list[numpat-1][1][1]
                    sub_date = newpat_list[numpat-1][1][4]

        elif type(newpat_list[numpat-1][1]) == dict:
            sub_name = newpat_list[numpat-1][1]['Patient Name']
            sub_adrema = newpat_list[numpat-1][1]['Patient ID']
            sub_date = newpat_list[numpat-1][1]['Study Date']

        print('Selected patient: %s: Exam date = %s - Tasks with all figures: %s' 
                   %(sub_name, sub_date, taskList))
        
    else:
        patient2report = []
        
    clear_output()
    
    return patient2report 
    

In [ ]:
def select_task(task_list):
    """
    Function to select the task for reporting SPM results
    
    returns variable 'task'
    """
    
    print('Task(s) for evaluation: %s' % task_list)
    
    task = raw_input('Choose a task? ')
    if task in task_list:
        pass
    else:
        print('Please select one in the list!')
        
    return task;

In [ ]:
def file_len(fname):
    """
    Function for finding the number of outliers by counting the number of lines in fname
    input: fname, the outliers file in preproc 
    
    """
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    try:
        return i + 1
    except:
        return 0

In [ ]:
def get_image_dimensions(imagefile):
    """
    Helper function that returns the image dimentions

    :param: imagefile str (path to image)
    :return dict (of the form: {width:<int>, height=<int>, size_bytes=<size_bytes>)
    """
    
    from PIL import Image
    
    # Inline import for PIL because it is not a common library
    with Image.open(imagefile) as img:
        # Calculate the width and hight of an image
        width, height = img.size

    # calculat ethe size in bytes
    size_bytes = os.path.getsize(imagefile)

    return dict(width=width, height=height, size_bytes=size_bytes)

In [ ]:
def printlines(number):
    #print('_'*72)
    for i in range(number):
        print('')
    #print('_'*72)

In [ ]:
def print_summary(patient2report):
    """
    Print the patient summary info
    save the name for the PDF output
    
    """
    
    if type(patient2report[1]) == list:
        for item in patient2report[1][1]:
            if hasNumbers(patient2report[1][0]):
                sub_name = patient2report[1][3]
                sub_adrema = patient2report[1][0]
                sub_date = patient2report[1][2]    
            else:
                sub_name = patient2report[1][0]
                sub_adrema = patient2report[1][1]
                sub_date = patient2report[1][4]

    elif type(patient2report[1]) == dict:
        sub_name = patient2report[1]['Patient Name']
        sub_adrema = patient2report[1]['Patient ID']
        sub_date = patient2report[1]['Study Date']
        
    outputPDF = sub_name + ' ' + sub_adrema + ' ' + sub_date + ' - fMRI.pdf'

    printmd('**Patient = %(n)s, Adrema = %(a)s**' % {'n' : sub_name, 'a' : sub_adrema}, color = 'blue')
    #date_fMRI = patient2report[1][4]
    try: 
        datetimeobject = datetime.strptime(sub_date,'%Y%m%d')
        date_fMRI = datetimeobject.strftime('%m-%d-%Y')
    except:
        pass
    printmd('**Datum van fMRI onderzoek: %s**' % date_fMRI, color = 'blue')
    printmd('*Uitgevoerde paradigmas = %s*' % subTasks)

    printmd("**fMRI rapport voor de uitgevoerde paradigma's**")
    print(START + 'Algemene processing info' + END)
    print('Verwerkt met FSL, ANT en SPM op DeepLearningOne') 
    print('Verwerkingsdata in %s' % suboDir)

    print('Datasink %s' % subResultsN)
    print('')
    print(START + 'Deze figuren worden getoond per paradigma:' + END)
    print('1. Realignatie plot en EPI signaal plot')
    print('2. Artefact detectie plot')
    print('3. Level 1 Design matrix')
    print('5. Glasbrein activatie')
    print('6. Sneden met activatie (mogelijk meerdere)')
    print('7. Relevante LI curves')

    print('_'*70)
    
    return outputPDF

In [ ]:
def print_QA_figs(task):
    """
    Will sequentially output 
    - the realignment plot for task
    - the timeline of a voxel for task
    - the artefact analysis for task
    - glass brain activation for F-contrast for task
    - overlays of activation for tasks on anatomical volumes
    - relevant LI curves
    
    """
    
    outliers = num_outliers_dict[task]
    
    # define the images to be displayed
    realignmentImage = opj(subFiguresN, 'realignmentplot_' + task + '.png')
    
    for ffile in os.listdir(subFiguresN):
        if task in ffile and ffile.startswith('detrended'):
            detrendImage = opj(subFiguresN, ffile)
            
    errorVolumes = opj(subResultsN, 'preproc', 'plot.r' + subid + '_task-' + task + '_bold.png')
                       
    glassImage = opj(subFiguresN, 'GlassBrain_' + task + '.png')
                       
    designImage = opj(subFiguresN, 'DesignMatrix_' + task + '.png')
    
    ima1 = plt.imread(realignmentImage)
    ima2 = plt.imread(detrendImage)
    ima3 = plt.imread(errorVolumes)
    ima4 = plt.imread(glassImage)
    ima5 = plt.imread(designImage)

    figPrep = plt.figure(figsize=(16,20), constrained_layout=True)

    figPrep.suptitle('Q&A for %s' % task, fontsize=20)

    gs1 = figPrep.add_gridspec(nrows=3, ncols=7, left=0.00, right=1, wspace=-.10)

    fP_ax1 = figPrep.add_subplot(gs1[0, :])
    fP_ax1.imshow(ima1)
    fP_ax1.axis('off')

    fP_ax2 = figPrep.add_subplot(gs1[-2, :-3])
    fP_ax2.imshow(ima2)
    fP_ax2.axis('off')

    fP_ax3 = figPrep.add_subplot(gs1[-2, -3:])
    titleEV = 'Volume errors - outliers = ' + str(outliers)
    fP_ax3.set_title(titleEV, fontsize=14)
    fP_ax3.imshow(ima3)
    fP_ax3.axis('off')

    fP_ax2 = figPrep.add_subplot(gs1[-1, :-3])
    fP_ax2.imshow(ima4)
    fP_ax2.axis('off')

    fP_ax2 = figPrep.add_subplot(gs1[-1, -3:])
    fP_ax2.imshow(ima5)
    fP_ax2.axis('off')

In [ ]:
def plot_LI_curves(task):
    """
    function to plot all the LI curves decently
    
    """
    
    # check how many LI curves for the task
    
    licList = sorted([f for f in os.listdir(subFiguresN) if f.startswith('LI_' + task.upper())])
    lenLI = len(licList)
        
    # print a single LI curve larger and if more than one always in 2 columns
    
    if lenLI == 0:
        print('No LI-curves for %s' % task.upper())
    elif lenLI == 1:
        fig = plt.figure(figsize=(12,8))
        fig.suptitle("LI Curves for " + task.upper(), fontsize=20)
        plt.imshow(imageio.imread(opj(subFiguresN, licList[0])))
        #axarr[0, 0].set_title('Frontal lobes', fontsize=14)
        plt.axis('off')
    elif lenLI > 1:
        # create an emty image is the number of LI-curves is unequal
        if lenLI%2 == 1:
            img = Image.new("RGB", (256, 256), (255, 255, 255))
            newLI = 'LI_' + task.upper() + '_ZZZZ.png'
            img.save(opj(subFiguresN, newLI))
            licList.append(newLI)
            lenLI += 1  
        # plot now an even number of images in 2 columns
        gridN = int((lenLI+1)/2)
        fig, axarr = plt.subplots(2,gridN,figsize=(16,12))
        fig.suptitle("LI Curves for " + task.upper(), fontsize=20)
        for figNum in range(lenLI):
            if figNum < gridN:
                rowNum = 0
                colNum = figNum
            else:
                rowNum = 1
                colNum = figNum - gridN

            image = imageio.imread(opj(subFiguresN, licList[figNum]))
            axarr[rowNum, colNum].axis('off')    
            axarr[rowNum, colNum].imshow(image)

In [ ]:
def plot_LIs(task):
    """
    Function to plot the LI curves to screen
    
    """
    
    countLI = 0
    LIList = []
    for ff in os.listdir(opj(subFiguresN)):
        if ff.startswith('LI_' + task.upper()):
            countLI += 1
            LIList.append(ff)
            
    fig = plt.figure()
    fig.set_size_inches(15, 30)
    
    for i, ff in enumerate(sorted(LIList)):
        img = mpimg.imread(opj(subFiguresN, ff))
        a = fig.add_subplot(len(LIList), 1, i+1)
        plt.axis('off')
        plt.imshow(img)
        region = ff.split('.')[0].split('_')[-1]
        a.set_title(task + ' ' + region)

In [ ]:
def print_report_old(task):
    """
    Will sequentially output 
    - the realignment plot for task
    - the timeline of a voxel for task
    - the artefact analysis for task
    - glass brain activation for F-contrast for task
    - overlays of activation for tasks on anatomical volumes
    - relevant LI curves
    
    """
    
    #printmd('***Rapport voor taak %s.***' % task.upper())

    display(Image(opj(subFiguresN, 'realignmentplot_' + task + '.png')))

    for ffile in os.listdir(subFiguresN):
        if task in ffile and ffile.startswith('detrended'):
            plotfile = ffile

    display(Image(opj(subFiguresN, plotfile)))

    display(SVG(filename = opj(subResultsN, 'preproc', 'plot.r' + subid + '_task-' + task + '_bold.svg')))
    
    outliers = num_outliers_dict[task]
    
    printmd('*Aantal uitgesloten EPI volumes met belangrijk artefact in taak %s = %s*' % (task.upper(), str(outliers)))
    
    printlines(3)
    
    def display_overlay(newtask):
        display(Image(opj(subFiguresN, 'GlassBrain_' + newtask + '.png')))
        display(Image(opj(subFiguresN, 'DesignMatrix_' + newtask + '.png')))
        
        printmd('**Designmatrix en glasbrein activatie voor %s**' % newtask.upper())
        
        printlines(3)
        
        overlay_list = []
        for ffile in os.listdir(subFiguresN):
            if newtask in ffile.lower() and ffile.startswith('overlay'):
                overlay_list.append(ffile)

        for overlayimage in overlay_list:        
            parts = overlayimage.split('_')
            if 'tra' in parts: direction = 'Axiaal'
            if 'cor' in parts: direction = 'Coronaal'
            if 'sag' in parts: direction = 'Sagittaal'
            if 'anat' in parts: bckgrnd = 'T1-MPRAGE'
            if 'epi' in parts: bckgrnd = 'EPI'
            if 'spmT' in parts: contrast = 'SPM-T'
            if 'spmF' in parts: contrast = 'SPM-F'

            overlay_title = newtask.upper() + ' ' + contrast + ' contrast op ' + bckgrnd + ' overlay.'
            display(Image(opj(subFiguresN, overlayimage)))

            printmd('***%s, neurologische conventie (links = links).***' % overlay_title, color = 'blue')
            
    if task != 'enconew':
        display_overlay(task)
    else:
        # we only show the ER results of enconew
        display_overlay(task)

    for ff in os.listdir(opj(subFiguresN)):
        if ff.startswith('LI_' + task.upper()):
            display(Image(opj(subFiguresN, ff), height=5)) 


In [ ]:
def display_overlay(task):
    
    overlay_list = []
    for ffile in os.listdir(subFiguresN):
        if task in ffile.lower() and ffile.startswith('overlay'):
            overlay_list.append(ffile)

    for overlayimage in overlay_list:
        #printlines(3)
        parts = overlayimage.split('_')
        if 'tra' in parts: direction = 'Axiaal (neurologische conventie!! LINKS = LINKS)'
        if 'cor' in parts: direction = 'Coronaal (neurologische conventie!! LINKS = LINKS)'
        if 'sag' in parts: direction = 'Sagittaal'
        if 'anat' in parts: bckgrnd = 'T1-MPRAGE'
        if 'epi' in parts: bckgrnd = 'EPI'
        if 'spmT' in parts: contrast = 'SPM-T'
        if 'spmF' in parts: contrast = 'SPM-F'

        overlay_title = task.upper() + '(' + contrast + ', ' + direction + ')'
        
        newImage = plt.imread(opj(subFiguresN, overlayimage))
        plt.figure(figsize = (20,30))
        plt.axis('off')
        plt.title('***%s.***' % overlay_title, fontsize = 20, y=.02, color="white")
        plt.imshow(newImage)#, aspect='auto')

In [ ]:
def print_report(task):
    """
    Will sequentially output 
    - the realignment plot for task
    - the timeline of a voxel for task
    - the artefact analysis for task
    - glass brain activation for F-contrast for task
    - overlays of activation for tasks on anatomical volumes
    - relevant LI curves
    
    """

    print_QA_figs(task)
    
    printlines(3)
    
    display_overlay(task)

    plot_LI_curves(task)


In [ ]:
def show_subjects():
    """
    function to display a tabel of the subjects in /data/Patients/SubjectInfoFiles/subjects.tsv
    
    """

    print(' #   Subject                                  Adrema          Accession  Date Exam  Referral                                           fMRI tasks                                                       DTI     F-map     Figs    Report')
    print('-'*220)
    with open(opj(iDir, 'subjects.tsv'), 'rt') as tsvFile:
        reader = csv.reader(tsvFile, delimiter='\t')
        #sortedLines = sorted(reader, key=operator.itemgetter(3))
        idx = 0
        for row in reader:
            idx = idx + 1
            row = [idx] + row
            print(' {:<3} {:<40} {:<15} {:<10} {:<10} {:<50} {:<65} {:<8} {:<8} {:<8} {:<8}'.format(*row))

In [ ]:
def update_subjects():
    """
    Function to update certain parameters in /data/Patients/SubjectInfoFiles/subjects.tsv
    
    """
    
    show_subjects()
    
    with open(opj(wDir, 'SubjectInfoFiles', 'subjects.tsv'), 'rt') as tsv_file:
        reader = csv.reader(tsv_file, delimiter='\t')
        lines = list(reader)

    rows = len(lines) 
    
    try:
        answer = int(input('\nWelke lijn wilt u aanpassen?:'))-1
        print(lines[answer])
        
        param2Change = {'Referral':4, 'Figs':8, 'Report':9}

        if answer in range(rows) and answer != 0:
            
            item = input('\nWelke waarde wil je veranderen? (kies uit: %s): ' % (', '.join(str(i) for i in list(param2Change.keys()))))
            if item in param2Change.keys():
                position = param2Change[item]
                print("\nHuidige waarde variable '%s' = %s" % (item, lines[answer][position]))
                newvalue = input("\nNieuwe waarde voor variabele '%s' = " % item)
                lines[answer][position] = newvalue
                
                print(lines[answer])
                
                with open(opj(wDir, 'SubjectInfoFiles', 'newsubjects.tsv'), 'w') as tsv_file:
                    writer = csv.writer(tsv_file, delimiter='\t')
                    writer.writerows(lines)
                
                shutil.copy(opj(wDir, 'SubjectInfoFiles', 'subjects.tsv'), opj(wDir, 'SubjectInfoFiles', 'subjects.tsv.backup'))
                shutil.copy(opj(wDir, 'SubjectInfoFiles', 'newsubjects.tsv'), opj(wDir, 'SubjectInfoFiles', 'subjects.tsv'))
                os.remove(opj(wDir, 'SubjectInfoFiles', 'newsubjects.tsv'))
                
                clear_output()
                
                print('Dit is the aangepaste subjects.tsv file:\n')
                show_subjects()
                
            else:
                print('Deze parameter bestaat niet!')

        else:
            print('Deze rij bestaat niet (of is hoofding)!')

    except:
        print('Fout! Duid een lijn aan met een nummer!')

In [ ]:
def svgtopng(task):
    """
    Convert the error volume svg to png format
    
    """
    
    svgFile = opj(subResultsN, 'preproc', 'plot.r' + subid + '_task-' + task + '_bold.svg')
    pngFile = opj(subResultsN, 'preproc', 'plot.r' + subid + '_task-' + task + '_bold.png')
    
    svg2png(url=svgFile, write_to=pngFile)
    
    print('converted %s to %s' % (svgFile, pngFile))

# Patient summary information

In [ ]:
patient_list = subjects4report()

In [ ]:
patient2report = select_patient4report(patient_list)

In [ ]:
# Here we define some useful variables for reporting
subid = patient2report[0]
subDir = opj(wDir, patient2report[0])
suboDir = opj(oDir, patient2report[0])
subResults = opj(oDir, 'results_' + patient2report[0])
subResultsN = opj(oDir, 'results_' + patient2report[0] + '-N')
subFigures = opj(subResults, 'figures')
subFiguresN = opj(subResultsN, 'figures')
taskList = list(patient2report[2].keys())
subTasks = [x.split('-N')[0] for x in taskList]

In [ ]:
# Try to publish into PDF, create the necessary dirs
ippub_outputpath = subFiguresN + '/ippub_output/'
if os.path.isdir(ippub_outputpath) == False:
    print('Creating dir %s' % ippub_outputpath)
    os.mkdir(ippub_outputpath)
else:
    print('%s exists.' % ippub_outputpath)

time.sleep(2)

clear_output()

In [ ]:
num_outliers_dict = {}
for task in subTasks:
    file_id = 'art.r' + subid + '_task-' + task + '_bold_outliers.txt'
    num_outliers = file_len(opj(subResultsN, 'preproc', file_id))
    num_outliers_dict[task] = num_outliers

In [ ]:
outputPDF = print_summary(patient2report)

In [ ]:
newTaskList = [f.split('-')[0].upper() for f in taskList if f.endswith('N')]
taskString = ', '.join(newTaskList)

In [ ]:
task = input('Report for task: ')
clear_output()

In [ ]:
display(Markdown("""# Rapport voor %s""" % task))

In [ ]:
if task.upper() in newTaskList:
    print_QA_figs(task)
    display_overlay(task)
    plot_LI_curves(task)
else:
    print('%s.upper not in task list!')

In [ ]:
task = input('Report for task: ')
clear_output()

In [ ]:
display(Markdown("""# Rapport voor %s""" % task))

In [ ]:
if task.upper() in newTaskList:
    print_QA_figs(task)
    display_overlay(task)
    plot_LI_curves(task)
else:
    print('%s.upper not in task list!')

In [ ]:
task = input('Report for task: ')
clear_output()

In [ ]:
display(Markdown("""# Rapport voor %s""" % task))

In [ ]:
if task.upper() in newTaskList:
    print_QA_figs(task)
    display_overlay(task)
    plot_LI_curves(task)
else:
    print('%s.upper not in task list!')

In [ ]:
task = input('Report for task: ')
clear_output()

In [ ]:
display(Markdown("""# Rapport voor %s""" % task))

In [ ]:
if task.upper() in newTaskList:
    print_QA_figs(task)
    display_overlay(task)
    plot_LI_curves(task)
else:
    print('%s.upper not in task list!')

In [ ]:
task = input('Report for task: ')
clear_output()

In [ ]:
display(Markdown("""# Rapport voor %s""" % task))

In [ ]:
if task.upper() in newTaskList:
    print_QA_figs(task)
    display_overlay(task)
    plot_LI_curves(task)
else:
    print('%s.upper not in task list!')

In [ ]:
task = input('Report for task: ')
clear_output()

In [ ]:
display(Markdown("""# Rapport voor %s""" % task))

In [ ]:
if task.upper() in newTaskList:
    print_QA_figs(task)
    display_overlay(task)
    plot_LI_curves(task)
else:
    print('%s.upper not in task list!')

# Additionele commentaar

In [ ]:
print('Standaard drempels voor spmT in UZG:')
print(drempels_dict)
print('')

answer = input('Standaard spmT drempels en correcties? [J]/N')
if answer == '' or answer == 'J':
    clear_output()
    for task in taskList:
        task = task.split('-')[0]
        if task == 'enconew': print('Gebruikte spmT drempel voor %s = %s' % (task.upper(), drempels_dict['enco_new']))
        if task == 'encoold': print('Gebruikte spmT drempel voor %s = %s' % (task.upper(), drempels_dict['enco_old']))
        if task == 'read': print('Gebruikte spmT drempel voor %s = %s' % (task.upper(), drempels_dict['read']))
        if task == 'wgen': print('Gebruikte spmT drempel voor %s = %s' % (task.upper(), drempels_dict['wgen']))
        if task == 'handen': print('Gebruikte spmT drempel voor %s = %s' % (task.upper(), drempels_dict['handen']))
        if task == 'voeten': print('Gebruikte spmT drempel voor %s = %s' % (task.upper(), drempels_dict['voeten']))
        if task == 'mond': print('Gebruikte spmT drempel voor %s = %s' % (task.upper(), drempels_dict['mond']))
        if task == 'visus': print('Gebruikte spmT drempel voor %s = %s' % (task.upper(), drempels_dict['visus']))    
    print('')
    print('Geen FWE noch Bonferroni correcties')
elif answer == 'N':
    clear_output()
    for task in taskList:
        task = task.split('-')[0]
        comment = input('Gebruikte spmT drempel voor %s: ' % task.upper())
    print('')
    answer = input('Correcties: [J]/N')
    if answer == '' or answer == 'N':
        print('Geen FWE noch Bonferroni correcties')
    elif answer == 'J':
        for task in taskList:
            corr = input('Correcties voor %s' % task)
else:
    print('Antwoord a.u.b. met J of N.')

comment = input('Extra commentaar: ')

In [ ]:
# Try to publish into PDF, create the necessary dir
ippub_outputpath = subFiguresN + '/ippub_output/'
if os.path.isdir(ippub_outputpath) == False:
    print('Creating dir %s' % ippub_outputpath)
    os.mkdir(ippub_outputpath)
else:
    print('%s exists.' % ippub_outputpath)
    
reportnotebook = '/home/rikachten/Dropbox/Notebooks/fMRI_UZG/fMRI Structured Report.ipynb'
oldnamePDF = '/home/rikachten/Dropbox/Notebooks/fMRI_UZG/fMRI Structured Report.pdf'
newnamePDF = '/home/rikachten/Dropbox/Notebooks/fMRI_UZG/fMRI_results/' + outputPDF

print('When this text disappears: save the notebook before running the last cells!!!!')

time.sleep(5)
clear_output()

In [ ]:
#!nbpublish -f latex_ipypublish_nocode -o {ippub_outputpath} -pdf {reportnotebook}

In [ ]:
%%capture
!jupyter nbconvert --no-input --to pdf '{reportnotebook}'

In [ ]:
os.rename(oldnamePDF, newnamePDF)

In [ ]:
update_subjects()